In [ ]:
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup as bs
import sqlite3

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# setup sqlite database
connectDB = sqlite3.connect('news.sqlite')
cur = connectDB.cursor()


In [ ]:
# scraping parameters
startyear = 2018
endyear = 2018
startmonth = 2
endmonth = 2
startday = 1
endday = 31

# looping through years, months, days
print("begin...")
for year in range(startyear,endyear+1):
  for month in range(startmonth,endmonth+1):
    for day in range(startday,endday+1):
      page = 1
      redirected = False

      # looping through every pages in index
      while True:

        # skip to next loop if page is redirected
        if redirected == True:
          break
        
        # trying until the request replied
        while True:
          try:
            concatedURL = 'https://www.jpnn.com/indeks?id=276&d='+str(day)+'&m='+str(month)+'&y='+str(year)+'&tab=all&page='+str(page)
            reqIndex = urllib.request.Request(concatedURL, headers={'User-Agent': 'Mozilla/5.0'})
            indeks = urllib.request.urlopen(reqIndex, context=ctx)
          except:
            print("retrying connection...")
            time.sleep(1)
            continue
          break
        
        # breaking the loop if redirected and pass redirected parameter to break parent loop
        if concatedURL != indeks.geturl(): 
          print(concatedURL+" is redirected to "+indeks.geturl())
          redirected = True
          break
        
        # selecting links in indeks
        indeks = indeks.read()
        parsedIndex = bs(indeks, 'html.parser')
        linkSelector = '#content-utama > div.kolom-kiri > div.border-box > div > ul > li > div.content-description > div > h2 > a'
        links = parsedIndex.select(linkSelector)

        # iterate every links
        for link in links:
          title = link.get_text()
          contentURL = urllib.parse.quote(link['href'], safe="http://")

          # same tricks as earlier request
          while True:
            try:
              reqContent = urllib.request.Request(contentURL, headers={'User-Agent': 'Mozilla/5.0'}) 
              content = urllib.request.urlopen(reqContent, context=ctx).read()
            except:
              print("retrying connection...")
              time.sleep(1)
              continue
            break
          
          # obtaining location data
          parsedContent = bs(content, 'html.parser')
          locationSelector = '#content-utama > div.kolom-kiri > div.border-box > div > div.content-scroll > div.page-content > div > p > a'
          locationData = parsedContent.select(locationSelector)

          # if not available, assign "None" value
          try: 
            location = locationData[0].contents[0]
          except: 
            location = "None"
            pass
          
          # obtaining first page content
          fpSelector = '#content-utama > div.kolom-kiri > div.border-box > div > div.content-scroll > div.page-content > div > p'
          fpData = parsedContent.select(fpSelector)

          # if no content, use None
          try:
            fp = fpData[0].get_text()
          except:
            fp = None

          date = str(year)+'-'+str(month)+'-'+str(day)
          link = link['href']
          
          # insert new row into database
          cur.execute('INSERT INTO Source (title, firstparagraph, date, location, link) VALUES (?, ?, ?, ?, ?)', (title, fp, date, location, link))
          connectDB.commit()

        print(str(day)+'/'+str(month)+'/'+str(year)+" page "+str(page)+' was parsed')

        # find whether there is more pages
        nextButton = parsedIndex.find_all(rel='next') 
        if nextButton == []:
          break
        else:
          page += 1
                

In [ ]:
# DATA PROCESSING

# edit date
from datetime import date as cd

cur2 = connectDB.cursor()
cur3 = connectDB.cursor()

cur2.execute('SELECT date,link FROM Source')
count = 0

print("begin...")

for row in cur2:
    link = row[1]
    time = row[0].split("-")
    year = int(time[0])
    month = int(time[1])
    day = int(time[2])
    try:
        date = cd(year, month, day).isoformat()
        cur3.execute('UPDATE Source SET date = ? WHERE link = ?', (date, link))
        connectDB.commit()
    except ValueError:
        # deleting date that out of range caused by parsing error 
        # (e.g. Feb 30th news is impossible)
        cur3.execute('DELETE FROM Source WHERE link = ?', (link,))
        connectDB.commit()
        print(link + "deleted")


    

In [ ]:
# substract jpnn.com and google analytics tag from content
import sqlite3

connectDB = sqlite3.connect('news.sqlite')
findjpnn = connectDB.cursor()
findjpnn.execute("SELECT firstparagraph,link FROM Source")

recontent = connectDB.cursor()

checkword = ('jpnn.com - ','JPNN.Com - ','JPNN.com - ','jpnn.com - ','JPNN.com - ','JPNN.com--','jpnn.com, ','JPNN.com – ','JPNN.com- ','JPNN.com – ','JPNN.Com – ','JPNN.Com – ','JPNN.Com – ','JPNN.com — ','jpnn.com - ','jpnn.com- ','jpnn.com -','JPNN.com ','JPNN.com - ','JPNN.com — ','JPNN.Com - ')
gtag = "googletag.cmd.push(function() { googletag.display('div-gpt-ad-1556314520901-0'); });"

count = 0

print("begin loop...")
for row in findjpnn:
    content = row[0]
    link = row[1]

    if (content == "") or (content == None): 
        recontent.execute("DELETE FROM Source WHERE link = ?",(link,))
        connectDB.commit()
        print(link+" is deleted")
    else:    
        for word in checkword:
            if (content.find(word) != -1) or (content.find(gtag) != -1):
                content = content.replace(word,"")
                content = content.replace(gtag,"")
                recontent.execute("UPDATE Source SET firstparagraph = ? WHERE link = ?",(content, link))
                connectDB.commit()
                update = True
            else:
                update = False
        if update == True: print(link+" is updated")
    
    # count += 1
    # if count == 401: break
print("done")


In [ ]:
# Attempt to get location data from content paragraph using regular expression
cur4 = connectDB.cursor()
cur5 = connectDB.cursor()

import re

cur4.execute('SELECT firstparagraph,link FROM Source')

count = 0


for row in cur4:
    locationdata = row[0]
    linkdata = row[1]
    locationdata = re.findall('^([A-Z]+.[A-Z]+)\s-',locationdata) or re.findall('^([A-Z]+.[A-Z]+)-',locationdata) or re.findall('^([A-Z]+.[A-Z]+)\s\W',locationdata)
    if locationdata == []: 
        cur5.execute('UPDATE Source SET location = ? WHERE link = ?', ('None',linkdata,))
        nolocation = True
        # print(linkdata)
        connectDB.commit()
    else:
        cur5.execute('UPDATE Source SET location = ? WHERE link = ?', (locationdata[0],linkdata,))
        connectDB.commit()
        print(locationdata[0]+" "+linkdata)
        
    # if nolocation == True: print(row[1]+" no location")

print("done")

In [ ]:
# delete if not uppercase (location not automatically determined)
locupper = connectDB.cursor()
locupper.execute("SELECT location,link FROM Source")
deleterow = connectDB.cursor()

import re

count = 0
for row in locupper:
    location = row[0]
    link = row[1]
    location = re.findall('^([A-Z][A-Z]+)',location)
    if location == []: 
        deleterow.execute("DELETE FROM Source WHERE link = ?",(link,))
        connectDB.commit()
        count += 1
    

In [24]:
print(str(count)+" row deleted")


7307 row deleted
